<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Predicting_Eye_Diseases_From_Fundus_Photographs_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_path = "/content/drive/My Drive/Eye Disease Research/train/"
test_path = "/content/drive/My Drive/Eye Disease Research/test/"

In [6]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [10]:
imgdata = ImageDataGenerator(validation_split=0.2, rescale=1.0/255.0)
train_batches = imgdata.flow_from_directory(train_path,subset='training', target_size=(512,496) ,
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 32)
test_batches = imgdata.flow_from_directory(test_path, target_size=(512,496) ,
                                           classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 16)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(512,496) ,
                                            classes = ['CNV','DME','DRUSEN','NORMAL'], batch_size = 32)

model = Sequential()
model.add(Conv2D(16, (3,3),activation='relu', padding='same', input_shape=(512,496,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3,3),activation='relu'))
model.add(Conv2D(32, (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3,3),activation='relu', padding='valid'))
model.add(Conv2D(64, (3,3),activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256, activation ='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation ='relu'))
model.add(Dense(4, activation='softmax'))

model.summary()

Found 32848 images belonging to 4 classes.
Found 1001 images belonging to 4 classes.
Found 8211 images belonging to 4 classes.
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 512, 496, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 256, 248, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 254, 246, 32)      4640      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 252, 244, 32)      9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 126, 122, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 124, 120, 64)      1849

In [0]:
from sklearn.utils import class_weight

In [12]:
mapp = {}
for e in train_batches.classes:
    if(e not in mapp):
        mapp[e]=1
    else:
        value = mapp[e]
        mapp[e] = value+1
print(mapp)

{0: 384, 1: 4507, 2: 6901, 3: 21056}


In [13]:
for ele in mapp:
    print("Value: %d is that times:" % ele, mapp[ele]/384)

Value: 0 is that times: 1.0
Value: 1 is that times: 11.736979166666666
Value: 2 is that times: 17.971354166666668
Value: 3 is that times: 54.833333333333336


In [0]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), train_batches.classes)

In [16]:
print(class_weights)

[21.38541667  1.82205458  1.18997247  0.3900076 ]


In [0]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit_generator(train_batches, steps_per_epoch=8, epochs=8,verbose=1, class_weight=class_weights, 
                    validation_data=valid_batches, validation_steps=8, shuffle=True)

Epoch 1/8
8/8 [==============================] - 124s 16s/step - loss: 327.9613 - accuracy: 0.5664 - val_loss: 1.2562 - val_accuracy: 0.6758
Epoch 2/8
8/8 [==============================] - 120s 15s/step - loss: 1.1639 - accuracy: 0.6523 - val_loss: 1.0252 - val_accuracy: 0.6875
Epoch 3/8
8/8 [==============================] - 120s 15s/step - loss: 0.9148 - accuracy: 0.7148 - val_loss: 1.0206 - val_accuracy: 0.5820
Epoch 4/8
8/8 [==============================] - 122s 15s/step - loss: 0.9213 - accuracy: 0.6797 - val_loss: 1.4593 - val_accuracy: 0.5664
Epoch 5/8
8/8 [==============================] - 120s 15s/step - loss: 0.8821 - accuracy: 0.6875 - val_loss: 0.7771 - val_accuracy: 0.6523
Epoch 6/8
8/8 [==============================] - 120s 15s/step - loss: 0.9761 - accuracy: 0.6172 - val_loss: 1.0444 - val_accuracy: 0.6367
Epoch 7/8
8/8 [==============================] - 119s 15s/step - loss: 1.0037 - accuracy: 0.5898 - val_loss: 0.8518 - val_accuracy: 0.6719
Epoch 8/8
8/8 [==========

In [19]:
acc = model.evaluate_generator(test_batches, steps=len(test_batches), verbose=1)


63/63 [==============================] - 377s 6s/step


In [21]:
print((acc[1] * 100))

24.97502565383911


In [22]:
print((acc))

[2.2379252910614014, 0.2497502565383911]
